<a target="_blank" href="https://colab.research.google.com/github/bettercodepaul/nixtla_intro_workshop/blob/main/Introduction_to_Nixtlaverse.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Introduction to Forecasting with Nixtla's Nixtlaverse

This notebook walks you through the very basics of forecasting time series with Nixtla's Nixtlaverse.

## Install and import necessary libraries

We use [Polars](https://docs.pola.rs/) for data wrangling, [Plotly](https://plotly.com/python/plotly-express/) for visualizations and Nixtla's [StatsForecast](https://nixtlaverse.nixtla.io/statsforecast/index.html) for basic time series forecasting.

In [ ]:
pip -q install statsforecast polars plotly

In [ ]:
# Download utility module
import urllib.request
import os.path
UTILS_URL = "https://github.com/bettercodepaul/nixtla_intro_workshop/raw/main/utilities.py"
urllib.request.urlretrieve(UTILS_URL, os.path.basename(UTILS_URL))

In [ ]:
import polars as pl
import plotly.express as px
from statsforecast import StatsForecast
from datetime import date
import utilities as bcxp_ts_utils

## Time Series

A time series is a collection of data points recorded or observed at consistent, regular intervals over time, such as hourly, daily, monthly, or yearly. This type of data captures how certain metrics or phenomena evolve over time, making it an essential tool for analyzing trends, forecasting future values, and understanding temporal patterns.

- **Population trends**
    - Example: The population of a city measured at the start of each year from 2000 to 2020.
- **Economic indicators**
    - Example: The national unemployment rate recorded monthly by a government agency.
- **Stock market data**
    - Example: The daily closing price of a company's stock for the past six months.
- **Production data**
    - Example: The number of cars produced in a factory each month over the last three years.
- **Sales performance**
    - Example: The weekly sales revenue of an online store during a calendar year.
- **Resource utilization**
    - Example: The hourly CPU usage of a server over 24 hours.
- **Energy consumption**
    - Example: The daily electricity consumption of a household over a year.
- **Sensor data**
    - Example: Hourly temperature readings from a weather station for one month.

## Initial Exploration of the data

The data for this walk through is simple monthly sales data from various countries.

In [ ]:
# load the dataset
Y_df = pl.read_parquet("https://github.com/bettercodepaul/nixtla_intro_workshop/raw/refs/heads/main/retail_sales.parquet")
# show a sample of 5 rows
Y_df.sample(5)

Nixtla follows a specific naming convention for the time series data.

- `unique_id`: an identifier to distinguish different time series in the same data set
- `ds`: the date or time column
- `y`: the actual value of the time series at that time

We can visualize the time series with Plotly.

In [ ]:
# This shows a line plot with the sales data for each country
# Plotly comes with useful interactive features: zoom, hover and trace isolation in the legend via click/double click
px.line(Y_df, x="ds", y="y", color="unique_id")

In [ ]:
# StatsForecast comes with a utility that can create a faceted plot for each time series in the dataset
StatsForecast.plot(Y_df, unique_ids=["Deutschland", "Frankreich", "Italien", "Grossbritannien", "Japan", "USA"], engine="plotly")

# Hands-on: Exploratory Data Dibbling

Explore the time series! What do you find interesting? Are there any obvious patterns? Are there any outliers? What could be the reasons?

In [ ]:
# you can either code here your own explorations or just use the interactive diagrams above

## Understanding the Structure and Influences of Time Series Data

There are fundamental truths that remain constant over time. For instance, physical laws govern natural processes, and basic economic principles, like the relationship between supply and demand, operate consistently. Additionally, controlled lab experiments, when repeated, yield the same results regardless of timing.

However, much of the data we analyze in reality is time-dependent. Long-term trends, such as the emergence of new technologies or the implementation of new regulations, evolve over broader periods. Meanwhile, short-term events like a strike or a marketing campaign can significantly impact data. Seasonal patterns, like spikes in retail sales during holidays, also create recurring fluctuations.

Human behavior often adds another layer of influence. For example, companies might increase sales figures at year-end using creative methods just to meet annual targets.

To understand time series data, we typically break it down into three components:

- **Trend**: The overall direction of the data over time (e.g., population growth or declining energy usage).
- **Seasonality**: Regular patterns that repeat within fixed intervals, like weekly sales cycles or holiday shopping spikes.
- **Residual**: Unpredictable variations or noise, caused by random events or minor anomalies.

The environment, including the day-night cycle or seasonal shifts, plays a significant role in shaping time series across all domains, as illustrated in the graphic below.

In [ ]:
# Decompose the time series data using a multiplicative model and monthly seasonality (this uses StatsModels seasonal_decompose under the hood)
df_decomposed = bcxp_ts_utils.decompose(Y_df, model="multiplicative", period=12)
df_decomposed.sample(5)

In [ ]:
# plot the components for a single time series
bcxp_ts_utils.plot_components(df_decomposed, unique_id="Japan")

In [ ]:
# plot the trend component for all time series in the dataset
px.line(df_decomposed, x="ds", y="trend", color="unique_id")

In [ ]:
# plot the seasonal component for a single time series in a bar polar plot
bcxp_ts_utils.plot_seasonality(df_decomposed, unique_id="Japan")

In [ ]:
# plot the seasonal components for all time series in the dataset
bcxp_ts_utils.plot_seasonalities(df_decomposed, facet_col_wrap=3)

## Forecasting using Nixtla's StatsForecast

Nixtla's StatsForecast package comes with a lot of classic forecasting algorithms. We won't go into the details of the different algorithms in this workshop. If you would like to know more details about them we highly recommend the freely available book ["Forecasting: Principles and Practice, the Pythonic Way"](https://otexts.com/fpppy/) by Rob Hyndman.

As you might have noticed, the data contains a strong seasonal component in each year.
This is caused by different cultural habits, business practices and laws in each country.

Most algorithms that support to model this seasonal component have to be given the seasonal length. As we have a yearly seasonality and monthly data the seasonal length will be 12, which means that every 12 time steps the same period is repeated.

We will use 4 models in this example:

- [`HistoricAverage`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#historicaverage): forecast the mean of all past observations
- [`SeasonalNaive`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#seasonalnaive): forecast the last value of the same period (e.g. the same month of the previous year)
- [`HoltWinters`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#holtwinters): an exponential smoothing model, that models trend and seasonality
- [`AutoARIMA`](https://nixtlaverse.nixtla.io/statsforecast/src/core/models.html#autoarima): THE classic time series model, that models trend and seasonality. The parameters for the underlying SARIMA model are automatically selected.

In [ ]:
from statsforecast.models import (
    HistoricAverage,
    SeasonalNaive,
    HoltWinters,
    AutoARIMA,
)
models = [
    HistoricAverage(),
    SeasonalNaive(season_length=12),
    HoltWinters(season_length=12),
    AutoARIMA(season_length=12),
]

In [ ]:
# initialize the StatsForecast object with the models and frequency
sf = StatsForecast(
    models=models,
    freq="1mo",
)

In [ ]:
# create a forecast for the next 48 months (4 years) using all available data and all models
forecasts_df = sf.forecast(df=Y_df, h=48)
forecasts_df.sample(5)

We visualize the forecasts of the different models for the different countries.

In [ ]:
sf.plot(Y_df, forecasts_df, engine="plotly")

## Hands-on: Eyeballed Forecast Analysis

Explore the forecasts! What do you find interesting? Are the models capable of reproducing the seasonal patterns? Does the model forecast a certain trend? Is there a model you would prefer over the others? Does the forecasted minimum value make sense for all models?

In [ ]:
# you can either code here your own explorations or just use the interactive diagrams above

Make another forecast, but now start at January 2018

In [ ]:
Y_df_before_2018 = Y_df.filter(pl.col("ds").dt.year().lt(2018))

In [ ]:
# use the code from above to create the forecast using Y_df_before_2018
# visualize the forecast using the code from above, but still us Y_df, so that you can compare the forecast with the actual values
# what do you observe? Does the forecast interpolation of the past trend look right? How did the trend actually develop?

## Forecast Validation

As demonstrated in the hands-on example, we can evaluate the accuracy of our model by comparing its forecasts for past periods (derived from "old data") to the actual values we already know today.

However, one of the most significant challenges in validating forecasts with this method is future leakage. It is crucial to ensure that the forecasts are not influenced by future data, as doing so would compromise the integrity of the evaluation. Unfortunately, in practice, completely avoiding future leakage is almost impossible. During the creation of the final forecasting system, many decisions must be made, and these decisions are often (unconsciously) influenced by information from the entire dataset.

Using a naive, random cross-validation scheme would definitely introduce future leakage. In such a scenario, the model would have access to data from the entire time period, allowing it to "learn" about future trends or events. This would lead to an overly optimistic cross-validation score that does not reflect the model's true performance on unseen data.

To mitigate this risk, we employ a specialized cross-validation scheme designed for time series data. This approach is often referred to as rolling cross-validation with a sliding window or expanding window. These techniques ensure a more robust evaluation of the model's predictive abilities across a wider range of temporal instances while maintaining critical properties:

1. the training data remains contiguous, adhering to the requirements of time series models, and
2. future data is systematically excluded from influencing the model during training.

By adhering to this method, we can arrive at a more realistic estimate of how well our model is likely to perform in practice.

In [ ]:
# visualization of a rolling cross-validation with an expanding window
from IPython.display import Image
Image(url="https://raw.githubusercontent.com/Nixtla/statsforecast/main/nbs/imgs/ChainedWindows.gif")

The `cross_validation` method from the `StatsForecast` class performs a rolling cross-validation with an expanding window by default and accepts the following arguments:

- `df`: The training data frame containing the historical data.
- `h (int)`: The forecast horizon, representing the number of steps into the future to be forecasted. For example, in our case, this is set to 24 months.
- `step_size (int)`: The number of time steps by which the window is shifted for each validation iteration. In other words, this controls how far the window moves forward during the cross-validation process.
- `n_windows(int)`: The number of validation windows used for cross-validation. In other words, this determines how many historical forecasting processes you want to evaluate.

In [ ]:
# this takes some time...
cv_df = sf.cross_validation(
    df=Y_df,
    h=24,
    step_size=24, # try step_size 12 as well -> overlapping windows
    n_windows=4
)

In [ ]:
# you can check the resulting windows of the expanding window validation
windows = cv_df.group_by("cutoff").agg(pl.col("ds").max()).sort("cutoff")
windows

In [ ]:
colors = px.colors.qualitative.Plotly # used to color the windows

# create a line plot of the world wide monthly sales
fig = px.line(Y_df.group_by("ds").agg(pl.col("y").sum()).sort("ds"), x="ds", y="y")

# add the windows as vertical rectangles to the plot
for idx, window in enumerate(windows.rows()):
    start = window[0]
    end = window[1]
    fig.add_vrect(x0=start, x1=end, fillcolor=colors[idx%len(colors)], opacity=0.2)
    fig.add_annotation(x=window[0], xshift=5, xanchor="left", text=f"Window {idx+1}<BR>{start:%m/%y} - {end:%m/%y}", align="left", font=dict(color="grey"), showarrow=False)

fig

## Hands-on: Validation Judgement

Time series validation presents unique challenges because you often don’t have enough data to capture all relevant patterns.

- Seasonal models, for instance, require at least two periods of data (and almost all our data generating processes have a yearly seasonality!)
- Long-term trends evolve slowly, and in a start-up, the data may only reflect steady growth, making it nearly impossible for the model to predict a decline if it has never encountered one.
- Furthermore, rare events like pandemics or financial crises occur infrequently, perhaps once a decade, leaving little precedent to learn from.

Look at the data and the windows. Will this validation scheme be representative? Can we improve? What are the trade-offs?

In [ ]:
# Room for your thoughts

## Forecast Evaluation

There are several error metrics we can use to evaluate the models based on the forecast from the rolling cross-validation.

- **Mean Absolute Error**: [`mae`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#mean-absolute-error-mae)
    - measures the forecasting accuracy using the absolute deviation $\left|\hat{y}_t - y_t\right|$
- **Root Mean Squared Error**: [`rmse`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#root-mean-squared-error)
    - measures the forecasting accuracy using the squared deviation $\left(\hat{y}_t - y_t\right)^2$
- **Bias**: [`bias`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#bias)
    - measures the forecasting bias using the deviation $\hat{y}_t - y_t$
- **Mean Absolute Percentage Error**: [`mape`](https://nixtlaverse.nixtla.io/utilsforecast/losses.html#mean-absolute-percentage-error)
    - measures the forecasting accuracy using the absolute *relative* deviation $\left|\frac{\hat{y}_t - y_t}{y_t}\right|$
    - you can use this for communication with stakeholders, but beware when comparing or choosing models
    - it is problematic when $y_t \approx 0$

In [ ]:
from utilsforecast.losses import mae, rmse, bias, mape

In [ ]:
def evaluate_cv(df, metric):
    models = [c for c in df.columns if c not in ('unique_id', 'ds', 'cutoff', 'y')]
    evals = metric(cv_df, models=models)
    pos2model = dict(enumerate(models))
    return evals.with_columns(
        best_model=pl.concat_list(models).list.arg_min().replace_strict(pos2model)
    ).with_columns(pl.selectors.float().round(2))

In [ ]:
evaluation_df = evaluate_cv(cv_df, rmse)
evaluation_df.head()

## Hands-on: Forecast Evaluation

Try the different error metrics. Is it always the same model that "wins"?

Compare with a different validation scheme (different `n_windows` or `step_size`) 

In [ ]:
# Room for your analysis

## Additional topics and Take home assignments


You can explore various topics we could not cover today!

- Try different transformations of the target value, e.g. using the Box-Cox-Transformation. See https://otexts.com/fpppy/nbs/05-toolbox.html#sec-ftransformations
- Try a fixed size for the training window in the cross-validation (e.g. `input_size=4` years) to get a sliding window.
- Explore all available models. See https://nixtlaverse.nixtla.io/statsforecast/index.html#models
- Explore all available metrics. See https://nixtlaverse.nixtla.io/utilsforecast/losses.html
- Check out how to include exogenuous regressors. See https://nixtlaverse.nixtla.io/statsforecast/docs/how-to-guides/exogenous.html
- Check out how to include multiple seasonalities. See https://nixtlaverse.nixtla.io/statsforecast/docs/tutorials/multipleseasonalities.html